In [1]:
import os
from SchemaSubsetter.Skalpel.LLM import OpenAIRequestLLM
from NlSqlBenchmark.NlSqlBenchmarkFactory import NlSqlBenchmarkFactory
from NlSqlBenchmark.NlSqlBenchmark import NlSqlBenchmark
from NlSqlBenchmark.SchemaObjects import (
    Schema, SchemaTable, TableColumn
)


/data/kluoma/skalpel-subsetting-evaluation/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NlSqlBenchmark.NlSqlBenchmarkFactory


/data/kluoma/skalpel-subsetting-evaluation/venv/lib/python3.10/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(


In [2]:
os.chdir("/data/kluoma/skalpel-subsetting-evaluation/")

In [3]:
llm = OpenAIRequestLLM(api_config_file="./.local/openai.json")

In [4]:
with open("./src/prompts/column_renaming.prompt") as f:
    prompt = f.read()
prompt

'You are a data integration specialist whos job it is to determine the contents and meaning of a legacy database table and columns.\nThe goal is to create a data dictionary containing succinct definitions and full english word representations of the schema tables and columns.\nYou accomplish this by reviewing the table schema, and the contents of the columns you are analyzing.\n\nHere is the table and column definition:\n{table_columns}\n\nHere are some sample values from the {column_name} column:\n{sample_values}\n\nYour first task is to create a succinct dictionary definition to be used in the data dictionary.\nYour second task is to create a ANSI SQL-compatible column full name that names the column with full english words.\n\nYou will provide your response as a json object.\n\nHere is an example response\n\n```json\n{{\n    "definition": "The f_name column contains the first name of the customer.",\n    "name": "first_name"\n}}\n```\n\nNow do your best to analyze the {column_name} 

In [5]:
bm_fact = NlSqlBenchmarkFactory()
bm = bm_fact.build_benchmark(benchmark_name="snails")


In [11]:
schema = bm.get_active_schema(database="NTSB")
table = schema.tables[0]
column = table.columns[15]
print(table.name, column.name)

AIRBAG BAGDAMSOURCE


In [12]:
bm.set_active_schema(database_name="NTSB")
bm.get_sample_values(table.name, column.name, 10)

[50, 60, 59, 97, 69, 8, 79, 9, 6, 7]

In [13]:
formatted_prompt = prompt.format(
    table_columns = table.as_ddl(),
    column_name = column.name,
    sample_values = str(bm.get_sample_values(table.name, column.name))
)
for line in formatted_prompt.split("\n"):
    print(line)

You are a data integration specialist whos job it is to determine the contents and meaning of a legacy database table and columns.
The goal is to create a data dictionary containing succinct definitions and full english word representations of the schema tables and columns.
You accomplish this by reviewing the table schema, and the contents of the columns you are analyzing.

Here is the table and column definition:
CREATE TABLE AIRBAG (
  CASEID INTEGER PRIMARY KEY,
  PSU INTEGER,
  CASENO INTEGER,
  CASENUMBER VARCHAR,
  CATEGORY INTEGER,
  VEHNO INTEGER PRIMARY KEY,
  BAGNO INTEGER PRIMARY KEY,
  BAGLOCATION INTEGER,
  BAGSTATUS INTEGER,
  BAGTYPE INTEGER,
  BAGDEPLOY INTEGER,
  BAGMALFUNCTION INTEGER,
  BAGFLAPSOPEN INTEGER,
  BAGFLAPSDAM INTEGER,
  BAGDAMAGE INTEGER,
  BAGDAMSOURCE INTEGER,
  PREVCRASH FLOAT,
  PRIORMAINT FLOAT,
  DEPLOYEVENT FLOAT,
  BAGDVTOTAL FLOAT,
  BAGDVLONG FLOAT,
  BAGDVLAT FLOAT,
  BAGDVENERGY FLOAT,
  BAGDVSPEED FLOAT,
  BAGDVBES FLOAT,
  BAGDVEST FLOAT,


In [14]:
response = llm.call_llm(
    prompt = formatted_prompt
)

/data/kluoma/skalpel-subsetting-evaluation/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'wire.westpoint.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [15]:
for line in response[0].split("\n"):
    print(line)

```json
{
    "definition": "The BAGDAMSOURCE column records the coded source or cause of damage to the airbag (e.g., impact, deployment, environmental factors).",
    "name": "airbag_damage_source_code"
}
```
